In [1]:
import pandas as pd
import os
from datetime import datetime as dt
from sqlalchemy import create_engine
import getpass


In [2]:
airbnb_files = os.listdir("Resources")
cols = ["date","id","name","neighbourhood","latitude","longitude","room_type","price","number_of_reviews","reviews_per_month"]

In [3]:
df = pd.DataFrame()
for file in airbnb_files:
    date = (file.split("(")[1].split(")")[0])
    temp = pd.read_csv("Resources/"+file)[cols[1:]]
    temp["date"] = dt.strptime(date, "%b %d %Y").strftime("%Y-%m-%d")
    df = pd.concat([df,temp[cols]])
    

ValueError: time data '1 Jan 2016' does not match format '%b %d %Y'

In [4]:
df.head()


""


In [5]:
df.dtypes

date                  object
id                     int64
name                  object
neighbourhood         object
latitude             float64
longitude            float64
room_type             object
price                  int64
number_of_reviews      int64
reviews_per_month    float64
dtype: object

In [6]:
df.to_csv("data/All_airbnb_listings.csv",sep=",",index=False)

In [26]:
df.head()

,date,id,name,neighbourhood,latitude,longitude,room_type,price,number_of_reviews,reviews_per_month
0,2016-04-05,689130,Exquisite private room w/sauna,Sullivan's Gulch,45.534838,-122.649000,Private room,100,54,1.25
1,2016-04-05,5794291,Private suites near it all- Room #6,Sullivan's Gulch,45.533373,-122.648236,Private room,100,34,3.71
2,2016-04-05,10885290,LOCATION NE : Queen Bunk Bed-TOP,Sullivan's Gulch,45.534529,-122.647962,Shared room,40,9,4.09
3,2016-04-05,4918034,Private suites near it all- Room #2,Sullivan's Gulch,45.533280,-122.647862,Private room,100,55,3.85
4,2016-04-05,5134256,Private suites near it all- Room #4,Sullivan's Gulch,45.533274,-122.646870,Private room,70,68,4.93


In [39]:
sorted_airbnb = df.sort_values(
    ["id", "date"], ascending=True).reset_index(drop=True)
sorted_airbnb.head(15)

,date,id,name,neighbourhood,latitude,longitude,room_type,price,number_of_reviews,reviews_per_month
0,2015-03-01,7893,Alberta Arts 2brm,King,45.561636,-122.655729,Entire home/apt,95,2,0.00
1,2015-05-12,7893,Alberta Arts 2brm,King,45.561636,-122.655729,Entire home/apt,95,2,0.00
2,2016-01-01,7893,North Williams 2brm Home,Piedmont,45.575479,-122.666777,Entire home/apt,175,2,0.03
3,2017-10-04,7893,North Williams 2brm Home,Piedmont,45.575479,-122.666777,Entire home/apt,175,2,0.02
4,2017-11-13,7893,North Williams 2brm Home,Piedmont,45.575479,-122.666777,Entire home/apt,175,2,0.02
5,2017-12-09,7893,North Williams 2brm Home,Piedmont,45.575479,-122.666777,Entire home/apt,175,2,0.02
6,2018-01-16,7893,North Williams 2brm Home,Piedmont,45.575479,-122.666777,Entire home/apt,175,2,0.02
7,2018-02-08,7893,North Williams 2brm Home,Piedmont,45.575479,-122.666777,Entire home/apt,175,2,0.02
8,2018-04-11,7893,North Williams 2brm Home,Piedmont,45.575479,-122.666777,Entire home/apt,175,2,0.02
9,2018-05-13,7893,North Williams 2brm Home,Piedmont,45.575479,-122.666777,Entire home/apt,175,2,0.02


In [44]:
reviews = 0
current_id = 0
review_col = []
for i,row in sorted_airbnb.iterrows():
    if sorted_airbnb["id"].iloc[i] != current_id:
        current_id = sorted_airbnb["id"].iloc[i]
        reviews = sorted_airbnb["number_of_reviews"].iloc[i]
    else:
        reviews = sorted_airbnb["number_of_reviews"].iloc[i] - sorted_airbnb["number_of_reviews"].iloc[i-1]       
    
    review_col.append(reviews)
    
# review_col

In [45]:
sorted_airbnb['reviews'] = review_col


In [51]:
p = getpass.getpass(prompt="Password: ")
rds_connection_string = f"postgres:{p}@localhost:5432/airbnb_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

Password: ········


In [52]:
sorted_airbnb.to_sql("airbnb_portland",con=engine,index=False, if_exists="replace")

In [53]:
engine.execute("SELECT * FROM airbnb_portland").fetchall()

[('2015-03-01', 7893, 'Alberta Arts 2brm', 'King', 45.561635987075434, -122.65572923241864, 'Entire home/apt', 95, 2, 0.0, 2),
 ('2015-05-12', 7893, 'Alberta Arts 2brm', 'King', 45.561635987075434, -122.65572923241864, 'Entire home/apt', 95, 2, 0.0, 0),
 ('2016-01-01', 7893, 'North Williams 2brm Home', 'Piedmont', 45.575478987075435, -122.66677723241864, 'Entire home/apt', 175, 2, 0.03, 0),
 ('2017-10-04', 7893, 'North Williams 2brm Home', 'Piedmont', 45.57547899, -122.66677720000001, 'Entire home/apt', 175, 2, 0.02, 0),
 ('2017-11-13', 7893, 'North Williams 2brm Home', 'Piedmont', 45.57547899, -122.66677720000001, 'Entire home/apt', 175, 2, 0.02, 0),
 ('2017-12-09', 7893, 'North Williams 2brm Home', 'Piedmont', 45.57547899, -122.66677720000001, 'Entire home/apt', 175, 2, 0.02, 0),
 ('2018-01-16', 7893, 'North Williams 2brm Home', 'Piedmont', 45.575478987075435, -122.66677723241864, 'Entire home/apt', 175, 2, 0.02, 0),
 ('2018-02-08', 7893, 'North Williams 2brm Home', 'Piedmont', 45.57